In [3]:
!pip install twitter==2.0a2
import pandas as pd
import twitter

In [4]:
bearer = 'AAAAAAAAAAAAAAAAAAAAANXkXgEAAAAA9btVv9h0f4Gx8r3D8bE7Mto0C84%3DvqFXOjroHrNOVZwxQUyOBtuo9M7hCgcmR3IMOZTBhi1oIaHyeV'
from twitter import OAuth2
auth = OAuth2(bearer_token=bearer)
api = twitter.Twitter(auth=auth,api_version="2",format="")

In [8]:
def req_to_df(req) :
    """
    Entrée : req un objet de type TwitterDictResponse de la libraire twitter venant d'une recherche de tweets
    Sortie : 
        df_tweets : un dataframe contenant des informations sur les tweets
        df_users : un dataframe contenant des informations sur les utilisateurs
    """
    df_tweets = pd.DataFrame(req['data']) #DataFrame des tweets
    df_users = pd.DataFrame(req['includes']['users']) #DataFrame des utilisateurs
    return(df_tweets,df_users)

In [11]:
def query_to_df(query,deb,fin,twitter_api) :
    """
    Entrée :
        query : un string contenant la query de la recherche pour plus d'info https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
        deb : la date de début de la recherche au format 'AAAA-MM-JJTHH:MM:SSZ'
        fin : la date de fin de la recherche au format 'AAAA-MM-JJTHH:MM:SSZ'
        twitter_api : un objet de type API de la libraire twitter
    Sortie :
        df_tweets : un dataframe des tweets (hors retweets) postés entre la date de début et de fin de la recherche selon la query indiquée
        df_users :  un dataframe des utilisateurs ayant posté des tweets entre la date de début et de fin de la recherche selon la query indiquée
    """
    request = twitter_api.tweets.search.recent(
        query=query,
        start_time = deb,
        end_time = fin,
        max_results = 100,
        expansions = 'author_id',
        params = {
            'tweet.fields' : 'created_at',
            'user.fields' : 'id,name,description,public_metrics'
        }
    )
    df_tweets,df_users = req_to_df(request)
    oldest = request['meta']['oldest_id']

    #L'utilisation de l'option until_id et since_id est incompatible avec l'utilisation de start_time et end_time, on va donc délimiter les tweets entre deux id
    request_old = twitter_api.tweets.search.recent(
        query=query,
        end_time = deb,
        max_results = 100,
    ) 
    fst = request_old['meta']['newest_id']

    for k in range(400) :
        request = twitter_api.tweets.search.recent(
            query=query,
            max_results = 100,
            since_id = fst,
            until_id = oldest,
            expansions = 'author_id',
            params = {
                'tweet.fields' : 'created_at',
                'user.fields' : 'id,name,description,public_metrics'
                }
        )
        if request['meta']['result_count'] == 0 :
            break
        df_t,df_u = req_to_df(request)
        df_tweets = pd.concat([df_tweets,df_t],ignore_index=True)
        df_users = pd.concat([df_users,df_u],ignore_index= True)
        oldest = request['meta']['oldest_id']

    return(df_tweets,df_users)

In [12]:
df1,df2 = query_to_df("#ARGCRO lang:fr -is:retweet",deb = '2022-12-13T18:45:00Z',fin = '2022-12-13T21:15:00Z',twitter_api=api)

In [ ]:
df1.to_json("tweets.json")
df2.to_json("users.json")